In [156]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors


In [157]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
user_vectors = pd.read_csv(r"D:\dataset\encoding\user_vectors.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")

Label encode username column so that usernames can be retrieved after K-Nearest Neighbours calculation

In [158]:
encoder = LabelEncoder()
final_vectors = user_vectors.copy()
x = encoder.fit(final_vectors["username"])
#print(encoder.classes_)
username_enc = x.transform(final_vectors["username"])
username_enc

#adding encoded usernames to new dataframe
final_vectors["username"] = username_enc

#pd.set_option('display.max_rows', None)
final_arrays = final_vectors.values
#final_vectors.loc[final_vectors["username"]==7537]
#get username from number
#encoder.inverse_transform([7537])
final_vectors

,username,experience,gender,generation,fav_anime_period,Slice,Dementia,Arts,Adventure,Vampire,Comedy,Shounen,Sports,Power,Military,Space,Mecha,Music,Supernatural,Mystery,Horror,Kids,Cars,Harem,School,Shoujo,Parody,Psychological,Action,Sci-Fi,Game,Fantasy,Martial,Romance,Ai,Life,Drama,of,Ecchi,Police,Historical,Super,Magic,Thriller,Seinen
0,7537,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1179,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,6556,3,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,5037,3,1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,3575,3,0,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9297,8352,3,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
9298,2558,3,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9299,4967,3,1,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9300,9113,2,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Compute K-Nearest Neighbours

In [165]:
#function that returns a list of indexes of most similar users in "final_arrays" array
def neighbours():
    user = np.array([7537, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    nc = NearestNeighbors(n_neighbors = 15, metric="cosine") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data
    n = train.kneighbors([user], return_distance = False)
    return n

sim_names = []
neighbours = neighbours()
for i in neighbours[0]: #loop through nested array
    #print(final_arrays[i][0])
    name = x.inverse_transform([final_arrays[i][0]])
    sim_names.append(name)

#build new dataframe of neighbours
neighbour_frame = pd.DataFrame()
for i in users.columns:
    neighbour_frame[i] = None

for name in sim_names:
    user_row = users.loc[users["username"]==name[0]]
    neighbour_frame = pd.concat([neighbour_frame, user_row])

pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 500)
neighbour_frame


#final_vectors.loc[final_vectors["username"]==7537]
#(final_vectors.columns)    

,user_id,username,user_completed,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
0,2255153,karthiga,49,Active,Female,1990,3391,Millenials,"['Comedy', 'Shoujo', 'Romance']",Classic
7633,108217,marix_yume,118,Active,Female,1991,5138,Millenials,"['Shoujo', 'Romance', 'Comedy']",Classic
4654,82687,nexi,149,Active,Female,1989,4784,Millenials,"['Comedy', 'Romance', 'Shoujo']",Classic
8295,79420,crusnikgirl,48,Active,Female,1993,3317,Millenials,"['Comedy', 'Shoujo', 'Romance']",Classic
7132,487787,zebrasXP,275,Active,Female,1993,5091,Millenials,"['Romance', 'Shoujo', 'Comedy']",Classic
3176,40806,T-chan,149,Active,Female,1986,3452,Millenials,"['Romance', 'Shoujo', 'Comedy']",Classic
4735,26197,sunsetglow,100,Active,Female,1991,3127,Millenials,"['Action', 'Comedy', 'Romance']",Classic
6621,6598,thinkgreen,238,Active,Female,1989,3764,Millenials,"['Romance', 'Comedy', 'Drama']",Classic
2246,119501,skeezzz16,88,Active,Female,1992,3432,Millenials,"['Comedy', 'Shoujo', 'Drama']",Classic
5922,86043,xDeathWishx,241,Active,Female,1994,5092,Millenials,"['Comedy', 'Romance', 'Supernatural']",Classic


Create anime recommendations

In [168]:

#Put anime that have most members at the top of each users "lists"
def recommendations(neighbours, media_type):
    #Get top rated TV shows or movies from each user
    suggestions = [] #anime_id's of recommended anime
    for i in neighbours["username"]:
        user_ratings = ratings.loc[ratings["username"]==i]
        tv_shows = user_ratings[user_ratings["type"]=="TV"]
        movies = user_ratings[user_ratings["type"]=="Movie"]

        if media_type == "TV":
            show_ratings = tv_shows
        else:
            show_ratings = movies
        
        #There may be 0 movies in the user's top 10 list in which case we skip that user's recommendationan
        if len(show_ratings) > 1:
            top_rated = show_ratings.iloc[0]["title"]
            suggestions.append(top_rated)
            #print(top_rated)

    if len(suggestions)==0:
        print("No movies to recommend :/")
    
    return suggestions

recommendations(neighbour_frame, "TV")
#ratings

['Kamisama Hajimemashita',
 'D.Gray-man',
 'Gakuen Utopia Manabi Straight!',
 'One Piece',
 'Bishoujo Senshi Sailor Moon SuperS',
 'Suzumiya Haruhi no Yuuutsu',
 'Rurouni Kenshin: Meiji Kenkaku Romantan',
 'ef: A Tale of Memories.',
 'Jyu Oh Sei',
 'D.Gray-man',
 'Aa! Megami-sama! (TV)',
 'Bokura ga Ita',
 'Ghost Hunt',
 'Chrome Shelled Regios',
 'Shuffle! Memories']

Evaluation of the recommended anime

In [161]:
#function that returns a list of indexes of most similar users in "final_arrays" array
""" def neighbours():
    #loop through all users in test set 
    user = np.array([7537,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0])
    nc = NearestNeighbors(n_neighbors = 5, metric="cosine") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data from training data
    n = train.kneighbors([user], return_distance=False)
    #return nested list. each list item is a list of "final_arrays" indexes of n similar users
    return n

sim_names = []
neighbours = neighbours()
for i in neighbours[0]: #loop through nested array
    #print(final_arrays[i][0])
    name = x.inverse_transform([final_arrays[i][0]])
    sim_names.append(name)

#build new dataframe of neighbours
neighbour_frame = pd.DataFrame()
for i in users.columns:
    neighbour_frame[i] = None

for name in sim_names:
    user_row = users.loc[users["username"]==name[0]]
    neighbour_frame = pd.concat([neighbour_frame, user_row])
neighbour_frame """

#include analysis of distance metric between neighbours
#(final_vectors.columns)    

' def neighbours():\n    #loop through all users in test set \n    user = np.array([7537,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0])\n    nc = NearestNeighbors(n_neighbors = 5, metric="cosine") #euclidean\n    #maybe add some weightings\n    train = nc.fit(final_arrays) #fit the model with the user vector data from training data\n    n = train.kneighbors([user], return_distance=False)\n    #return nested list. each list item is a list of "final_arrays" indexes of n similar users\n    return n\n\nsim_names = []\nneighbours = neighbours()\nfor i in neighbours[0]: #loop through nested array\n    #print(final_arrays[i][0])\n    name = x.inverse_transform([final_arrays[i][0]])\n    sim_names.append(name)\n\n#build new dataframe of neighbours\nneighbour_frame = pd.DataFrame()\nfor i in users.columns:\n    neighbour_frame[i] = None\n\nfor name in sim_names:\n    user_row = users.loc[users["username"]==name[0]]\n    neighbour_frame = pd.concat([nei

Function that converts user information from website into vector